A front-end to collect data, train and test the GNN for NASDAQ100 stock predictions

In [ ]:
from dataset.data import *
from models.gnn import *

Data Collection

In [ ]:
stock_path = '/Users/vivek/.cache/kagglehub/datasets/jacksoncrow/stock-market-dataset/versions/2/stocks'
meta_path  = '/Users/vivek/.cache/kagglehub/datasets/jacksoncrow/stock-market-dataset/versions/2/symbols_valid_meta.csv'
nasdaq100_path = '/Users/vivek/.cache/kagglehub/datasets/jacksoncrow/stock-market-dataset/versions/2/nasdaq_100.csv'

stock_data = NASDAQDataset(stock_path, meta_path, nasdaq100_path, "2014-01-01") # Collect all data after 2014-01-01
stock_data.read_data()

# Generate graphs for each day (from 2014-01-01 onwards)

for date in stock_data.data:
    G = stock_data.daily_graph_generator(date)

In [ ]:
# Read the gml files and create PyG data objects

data_path = '/Users/vivek/Documents/PhD/UIUC/Fall24/CS598/Project/GNN-Stock-Market-Prediction/dataset/graphs'
data_list = [os.path.join(data_path, file) for file in os.listdir(data_path)]

train_loader = gnn_data_obj(data_list[0:1000]) # train on first 1000 graphs
test_loader = gnn_data_obj(data_list[1000:]) # test on remaining graphs

Training

In [ ]:
# Initialize the StockGNN model and optimizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stock_model = StockGNN().to(device)
optimizer = torch.optim.Adam(stock_model.parameters(), lr=0.005, weight_decay=1e-5)

In [ ]:
# TRAIN!

train_loss, test_loss = train_gnn(stock_model, optimizer, train_loader, test_loader, 10)

In [ ]:
# Plot the validation MAPE

# plt.plot(train_loss, label = 'Training Loss')

plt.plot(test_loss, label = 'Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Percentage Error')
plt.show()